Presentation theme: Your target audience is non-technical people interested in using your findings to maximize their profit when selling their home.

- figure out 3 - 4 questions:
    - what are the top 5 features that are the most important in determining house prices
    - how much in average is the difference in houses with waterfronts and houses without
    - which area in kings county attracts higher prices and vice versa
    - how have average house prices faired over time
    
    
- clean data & feature engineering
    - duplicates
    - find missing values
    - find odd values (too high / negative values)
    - find outliers values (figure out what to do with them)
    - do we need to change datatype of any columns? to date? to int64
    - do we need to encode any categorical data?
    - any new variabes we can create or combine
    
- which columns to drop
    - id / yr_renovated
   
- visualization & basic stats
    - use correlation matrix (use np.triangle etc)
   
   
- linear regression - multiple
    - plot the coefficients in a graph
    - identify how adding each coefficient improves the R2
    - pick out the highest coefficients and plot simple regression / use reg coef to identify deeper to above ques
    - use p value


- understand the output of statsmodel


Questions
* "How did you pick the question(s) that you did?"
* "Why are these questions important from a business perspective?"
* "How did you decide on the data cleaning options you performed?"
* "Why did you choose a given method or library?"
* "Why did you select those visualizations and what did you learn from each of them?"
* "Why did you pick those features as predictors?"
* "How would you interpret the results?"
* "How confident are you in the predictive quality of the results?"
* "What are some of the things that could cause the results to be wrong?"

Data Issues Encountered:

Duplicate Data
    - id

Missing Data
    - waterfront 
    - view 
    - yr_renovated

Outliers
    - sqft_living
    - sqft_lot
    - sqft_lot15
    - bedroom
    
Adding New Columns

Incorrect data type
    - sqft_basement: treated as text as oppose to int64 / float64 due to '?'

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set_style('darkgrid')

pd.options.display.float_format = '{:,.2f}'.format

In [2]:
# importing data
url = 'https://raw.githubusercontent.com/Ftan91/Project-1---Kings-County-House-Dataset/master/kc_house_data.csv'
df = pd.read_csv(url, parse_dates = ['date'])

In [3]:
# check first 5 rows of dataset
display (df.head())

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,2014-10-13,"221,900.00",3,1.00,1180,5650,1.00,nan,0.00,...,7,1180,0.0,1955,0.00,98178,47.51,-122.26,1340,5650
1,6414100192,2014-12-09,"538,000.00",3,2.25,2570,7242,2.00,0.00,0.00,...,7,2170,400.0,1951,"1,991.00",98125,47.72,-122.32,1690,7639
2,5631500400,2015-02-25,"180,000.00",2,1.00,770,10000,1.00,0.00,0.00,...,6,770,0.0,1933,nan,98028,47.74,-122.23,2720,8062
3,2487200875,2014-12-09,"604,000.00",4,3.00,1960,5000,1.00,0.00,0.00,...,7,1050,910.0,1965,0.00,98136,47.52,-122.39,1360,5000
4,1954400510,2015-02-18,"510,000.00",3,2.00,1680,8080,1.00,0.00,0.00,...,8,1680,0.0,1987,0.00,98074,47.62,-122.05,1800,7503


In [4]:
# sanity check for any duplicates
df = df.drop_duplicates(subset = 'id')
print ('Number of rows before: 21597')
print ('Number of rows after dropping duplicates: {}'.format(df.shape[0]))
print ('Total number of rows dropped: {}'.format(21597 - df.shape[0]))

Number of rows before: 21597
Number of rows after dropping duplicates: 21420
Total number of rows dropped: 177


In [5]:
# define function to return number of unique data types and the list of data types for a single column
def datatype_search(column):
    x = df[column].unique()
    y = set([type(number) for number in x])
    z = len(y)
    return list(y), z

# define function to return a random number based on given list of choice, added with probability
def function(column):
    unique = df[df[column].notnull()][column].unique()
    prob = df[df[column].notnull()][column].value_counts(normalize = True)
    return np.random.choice(unique, 1, p= prob)[0]

In [6]:
# check for missing values, multiple data entries for each column, negative values and 
isnull = df.isnull().sum().sort_values(ascending = False)

for column in isnull.index:
    print ('Column name: {}'.format(column))
    print ('Number of null entries: {}'.format(df[column].isnull().sum()))
    print ('Number of unique datatype: {}'.format(datatype_search(column)[1]))
    print ('Data entry type: {}'.format(datatype_search(column)[0]))
    print ('Min: {}, Max: {}'.format(df[column].min(), format(df[column].max())))
    print ('\n')

Column name: yr_renovated
Number of null entries: 3804
Number of unique datatype: 1
Data entry type: [<class 'numpy.float64'>]
Min: 0.0, Max: 2015.0


Column name: waterfront
Number of null entries: 2353
Number of unique datatype: 1
Data entry type: [<class 'numpy.float64'>]
Min: 0.0, Max: 1.0


Column name: view
Number of null entries: 63
Number of unique datatype: 1
Data entry type: [<class 'numpy.float64'>]
Min: 0.0, Max: 4.0


Column name: sqft_lot15
Number of null entries: 0
Number of unique datatype: 1
Data entry type: [<class 'numpy.int64'>]
Min: 651, Max: 871200


Column name: date
Number of null entries: 0
Number of unique datatype: 1
Data entry type: [<class 'numpy.datetime64'>]
Min: 2014-05-02 00:00:00, Max: 2015-05-27 00:00:00


Column name: price
Number of null entries: 0
Number of unique datatype: 1
Data entry type: [<class 'numpy.float64'>]
Min: 78000.0, Max: 7700000.0


Column name: bedrooms
Number of null entries: 0
Number of unique datatype: 1
Data entry type: [<class

In [9]:
missing_values_column = ['yr_renovated', 'waterfront', 'view']

for column in missing_values_column:
    df[column] = df[column].map(lambda x: function(column) if pd.isnull(x) else x)
    
df['sqft_basement'] = df['sqft_basement'].map(lambda x: function('sqft_basement') if x == '?' else x)

In [11]:
df['sqft_basement'] = df['sqft_basement'].map(lambda x: int(float(x)))
df['sqft_total'] = df['sqft_above'] + df['sqft_basement']


In [ ]:
df.drop(df[df.bedrooms > 20].index, axis = 0, inplace = True)

In [ ]:
df2 = df[['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'house_age']]

In [ ]:
sns.pairplot(df2)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21420 entries, 0 to 21596
Data columns (total 22 columns):
id               21420 non-null int64
date             21420 non-null datetime64[ns]
price            21420 non-null float64
bedrooms         21420 non-null int64
bathrooms        21420 non-null float64
sqft_living      21420 non-null int64
sqft_lot         21420 non-null int64
floors           21420 non-null float64
waterfront       21420 non-null float64
view             21420 non-null float64
condition        21420 non-null int64
grade            21420 non-null int64
sqft_above       21420 non-null int64
sqft_basement    21420 non-null int64
yr_built         21420 non-null int64
yr_renovated     21420 non-null object
zipcode          21420 non-null int64
lat              21420 non-null float64
long             21420 non-null float64
sqft_living15    21420 non-null int64
sqft_lot15       21420 non-null int64
sqft_total       21420 non-null int64
dtypes: datetime64[ns](1), floa

In [13]:
# what is the best plot for ordinal and categorical. using pairplot for continuous only
# how to figure out zipcode? Okay to plot bar chart but for ML need to encode?
# how to figure out the house age thing?

# 7 to 8 github
# 8 to 10 viz / correlation matrix
# 10 to 12 regression
# 

In [14]:
df.zipcode.nunique()

70